### Download & extract PACS dataset (optional)

In [13]:
# Install deps (safe to re-run)
!pip -q install datasets pillow tqdm

import os, io
from PIL import Image
from tqdm import tqdm
from datasets import load_dataset

root = "/content/data/PACS"
os.makedirs(root, exist_ok=True)

ds = load_dataset("flwrlabs/pacs", split="train")

if "category" in ds.features:
    label_field = "category"
elif "class" in ds.features:
    label_field = "class"
elif "label" in ds.features:
    label_field = "label"
else:
    raise RuntimeError(f"Unexpected schema: {ds.features}")

def class_name(row):
    v = row[label_field]
    if isinstance(v, int):
        return ds.features[label_field].names[v]
    return str(v)

# Normalize domains → art_painting/cartoon/photo/sketch
def norm_domain(v: str):
    s = str(v).strip().lower().replace(" ", "_").replace("-", "_")
    if s in {"art_painting", "cartoon", "photo", "sketch"}:
        return s
    # map common variants just in case
    if s in {"artpainting", "art_paintings"}:
        return "art_painting"
    return s  # fallback (we'll skip unknowns below)

# Write images to /content/data/PACS/<domain>/<class>/<i>.jpg
for i, row in tqdm(enumerate(ds), total=len(ds)):
    dom = norm_domain(row["domain"])
    if dom not in {"art_painting", "cartoon", "photo", "sketch"}:
        continue  # skip anything weird
    cls = class_name(row)
    out_dir = os.path.join(root, dom, cls)
    os.makedirs(out_dir, exist_ok=True)

    img = row["image"]
    if not isinstance(img, Image.Image):
        # Some datasets provide bytes; convert to PIL.Image
        img = Image.open(io.BytesIO(img["bytes"])).convert("RGB")
    img.save(os.path.join(out_dir, f"{i}.jpg"), quality=95)


100%|██████████| 9991/9991 [00:15<00:00, 634.21it/s]


### Imports, constants, and seed

In [14]:
import torch, random, os, numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm
import numpy as np
import pandas as pd
import os

DATA_ROOT = "/content/data/PACS"
SOURCES   = ["art_painting", "cartoon", "photo"]
TARGET    = "sketch"
IMG_SIZE  = 224
BATCH_SIZE = 64
NUM_WORKERS = 2

# seed set for reproducibility
def set_seed(seed=1337):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1337)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [15]:
_to_rgb = transforms.Lambda(lambda im: im.convert("RGB"))

def make_loaders(data_root, img_size=224, batch_size=64, num_workers=2, sources=None, target=None):
    tfm_train = transforms.Compose([
        _to_rgb,
        transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.05),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ])
    tfm_eval = transforms.Compose([
        _to_rgb,
        transforms.Resize(256),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ])

    def load_domain(name, tfm):
        p = Path(data_root)/name
        assert p.exists(), f"Missing domain folder: {p}"
        return datasets.ImageFolder(str(p), transform=tfm)

    # Build datasets
    src_train, per_domain_eval = [], {}
    class_to_idx = None

    for d in sources:
        ds_tr = load_domain(d, tfm_train)
        ds_ev = load_domain(d, tfm_eval)
        if class_to_idx is None:
            class_to_idx = ds_tr.class_to_idx
        else:
            assert ds_tr.class_to_idx == class_to_idx, "Class mapping differs across domains."
        assert ds_ev.class_to_idx == class_to_idx
        src_train.append(ds_tr)
        per_domain_eval[d] = ds_ev

    target_eval = load_domain(target, tfm_eval)
    assert target_eval.class_to_idx == class_to_idx
    per_domain_eval[target] = target_eval

    # Loaders
    train_ds = ConcatDataset(src_train)
    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=True,
        persistent_workers=(num_workers > 0), drop_last=True
    )
    eval_loaders = {
        d: DataLoader(ds, batch_size=batch_size, shuffle=False,
                      num_workers=num_workers, pin_memory=True,
                      persistent_workers=(num_workers > 0))
        for d, ds in per_domain_eval.items()
    }

    num_classes = len(target_eval.classes)
    return train_loader, eval_loaders, num_classes, target_eval.classes

train_loader, eval_loaders, num_classes, classes = make_loaders(
    DATA_ROOT, IMG_SIZE, BATCH_SIZE, NUM_WORKERS, SOURCES, TARGET
)
print(f"Train size: {len(train_loader.dataset)} | Num classes: {num_classes}")
print("Domains loaded:", list(eval_loaders.keys()))
print("Classes:", classes)

Train size: 6062 | Num classes: 7
Domains loaded: ['art_painting', 'cartoon', 'photo', 'sketch']
Classes: ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']


In [7]:
def build_resnet50(num_classes: int):
    # pretrained ResNet-50, replace final layer
    m = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    m.fc = nn.Linear(m.fc.in_features, num_classes)
    return m

@torch.no_grad()
def evaluate(model, loaders, device):
    model.eval()
    acc = {}
    for dname, loader in loaders.items():
        correct, total = 0, 0
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            pred = logits.argmax(1)
            correct += (pred == y).sum().item()
            total   += y.numel()
        acc[dname] = correct / max(total, 1)
    return acc


def train_erm(
    sources,
    target,
    train_loader,
    eval_loaders,
    num_classes,
    epochs=20,
    lr=3e-4,
    wd=0.05,
    out_dir="outputs_erm",
    seed=1337,
    use_amp=True,
):
    np.random.seed(seed); torch.manual_seed(seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = build_resnet50(num_classes).to(device)

    opt   = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
    loss_fn = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=(use_amp and device.type=="cuda"))

    best_target = 0.0
    logs = []
    os.makedirs(out_dir, exist_ok=True)

    for ep in range(1, epochs+1):
        model.train()
        running_loss = 0.0
        seen = 0

        pbar = tqdm(train_loader, desc=f"Epoch {ep}/{epochs}", leave=False)
        for x, y in pbar:
            x, y = x.to(device), y.to(device)
            opt.zero_grad(set_to_none=True)

            with torch.cuda.amp.autocast(enabled=(use_amp and device.type=="cuda")):
                logits = model(x)
                loss = loss_fn(logits, y)

            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()

            running_loss += loss.item() * y.size(0)
            seen += y.size(0)

        sched.step()

        acc = evaluate(model, eval_loaders, device)
        src_accs = [acc[d] for d in sources]
        avg_src = float(np.mean(src_accs))
        worst_src = float(min(src_accs))

        # Log
        row = {
            "epoch": ep,
            "train_loss": running_loss / max(seen, 1),
            "target_acc": acc[target],
            "avg_source_acc": avg_src,
            "worst_source_acc": worst_src,
        }
        for d in sources + [target]:
            row[f"acc_{d}"] = acc[d]
        logs.append(row)

        print(f"[Ep {ep:02d}] loss={row['train_loss']:.4f} | "
              f"tgt({target})={acc[target]:.3f} | src_avg={avg_src:.3f} | worst_src={worst_src:.3f}")

        # Save best-by-target
        if acc[target] > best_target:
            best_target = acc[target]
            torch.save(model.state_dict(), os.path.join(out_dir, "best_model.pt"))

    df = pd.DataFrame(logs)
    df.to_csv(os.path.join(out_dir, "training_log.csv"), index=False)
    print(f"\nBest target ({target}) accuracy: {best_target:.3f}")
    return df


In [ ]:
# Uses the loaders you already created:
# train_loader, eval_loaders, num_classes, classes = make_loaders(...)

df_logs = train_erm(
    sources=SOURCES,
    target=TARGET,
    train_loader=train_loader,
    eval_loaders=eval_loaders,
    num_classes=num_classes,
    epochs=20,
    lr=3e-4,
    wd=0.05,
    out_dir="outputs_erm",
    seed=1337,
    use_amp=True,   # set False if you hit AMP issues
)

# Peek last few rows
df_logs.tail()


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 156MB/s]
/tmp/ipython-input-2102803089.py:44: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(use_amp and device.type=="cuda"))
Epoch 1/20:   0%|          | 0/94 [00:00<?, ?it/s]/tmp/ipython-input-2102803089.py:60: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(use_amp and device.type=="cuda")):


[Ep 01] loss=0.4052 | tgt(sketch)=0.711 | src_avg=0.967 | worst_src=0.958


[Ep 02] loss=0.1278 | tgt(sketch)=0.582 | src_avg=0.976 | worst_src=0.965


[Ep 03] loss=0.0820 | tgt(sketch)=0.663 | src_avg=0.985 | worst_src=0.980


...

[Ep 17] loss=0.0044 | tgt(sketch)=0.728 | src_avg=1.000 | worst_src=1.000


[Ep 18] loss=0.0039 | tgt(sketch)=0.736 | src_avg=1.000 | worst_src=1.000


[Ep 19] loss=0.0029 | tgt(sketch)=0.728 | src_avg=1.000 | worst_src=1.000


[Ep 20] loss=0.0041 | tgt(sketch)=0.729 | src_avg=1.000 | worst_src=1.000

Best target (sketch) accuracy: 0.736


,epoch,train_loss,target_acc,avg_source_acc,worst_source_acc,acc_art_painting,acc_cartoon,acc_photo,acc_sketch
15,16,0.003183,0.729448,1.0,1.0,1.0,1.0,1.0,0.729448
16,17,0.004430,0.728430,1.0,1.0,1.0,1.0,1.0,0.728430
17,18,0.003913,0.735811,1.0,1.0,1.0,1.0,1.0,0.735811
18,19,0.002883,0.728175,1.0,1.0,1.0,1.0,1.0,0.728175
19,20,0.004060,0.729193,1.0,1.0,1.0,1.0,1.0,0.729193


In [ ]:
import pandas as pd

log_path = "outputs_erm/training_log.csv"
df = pd.read_csv(log_path)

best_idx = df["target_acc"].idxmax()
best = df.iloc[best_idx]

summary = {
    "best_epoch": int(best["epoch"]),
    "target_sketch_acc": round(float(best["target_acc"]), 4),
    "avg_source_acc": round(float(best["avg_source_acc"]), 4),
    "worst_source_acc": round(float(best["worst_source_acc"]), 4),
}
for d in SOURCES + [TARGET]:
    summary[f"{d}_acc"] = round(float(best[f"acc_{d}"]), 4)

summary


{'best_epoch': 18,
 'target_sketch_acc': 0.7358,
 'avg_source_acc': 1.0,
 'worst_source_acc': 1.0,
 'art_painting_acc': 1.0,
 'cartoon_acc': 1.0,
 'photo_acc': 1.0,
 'sketch_acc': 0.7358}

## IRMv1

In [20]:

_to_rgb = transforms.Lambda(lambda im: im.convert("RGB"))

def make_per_domain_train_loaders(data_root, img_size=224, batch_size=12, num_workers=2, sources=None):
    # same train transform as before
    tfm_train = transforms.Compose([
        _to_rgb,
        transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.1,0.1,0.1,0.05),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ])
    loaders = {}
    class_to_idx = None
    for d in sources:
        ds = datasets.ImageFolder(str(Path(data_root)/d), transform=tfm_train)
        if class_to_idx is None:
          class_to_idx = ds.class_to_idx
        else:
          assert ds.class_to_idx == class_to_idx
        loaders[d] = DataLoader(ds, batch_size=batch_size, shuffle=True,
                                num_workers=num_workers, pin_memory=True,
                                persistent_workers=(num_workers>0), drop_last=True)
    return loaders


In [17]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from pathlib import Path

IMG_SIZE_IRM = 224       # <= reduce from 224
BATCH_IRM    = 12        # <= try 8 if still OOM
NUM_WORKERS  = 0         # <= safer in Colab
_to_rgb = transforms.Lambda(lambda im: im.convert("RGB"))

def make_per_domain_train_loaders(data_root, sources):
    tfm_train = transforms.Compose([
        _to_rgb,
        transforms.RandomResizedCrop(IMG_SIZE_IRM, scale=(0.7, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.1,0.1,0.1,0.05),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ])
    loaders = {}
    class_to_idx = None
    for d in sources:
        ds = datasets.ImageFolder(str(Path(data_root)/d), transform=tfm_train)
        if class_to_idx is None: class_to_idx = ds.class_to_idx
        else: assert ds.class_to_idx == class_to_idx
        loaders[d] = DataLoader(
            ds, batch_size=BATCH_IRM, shuffle=True,
            num_workers=NUM_WORKERS, pin_memory=True,
            persistent_workers=False, drop_last=True
        )
    return loaders


In [21]:
import torch, numpy as np, os, random, itertools
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm
import pandas as pd
from torch.optim.swa_utils import AveragedModel, SWALR

def build_resnet50(num_classes: int):
    m = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    # freeze all, then unfreeze layer4+layer3 first; we’ll unfreeze layer2 later
    for p in m.parameters(): p.requires_grad = False
    for p in m.layer4.parameters(): p.requires_grad = True
    for p in m.layer3.parameters(): p.requires_grad = True
    m.fc = nn.Linear(m.fc.in_features, num_classes)
    return m

def set_bn_eval(m):
    if isinstance(m, torch.nn.modules.batchnorm._BatchNorm):
        m.eval()

def irm_penalty(loss, dummy_w):
    g = torch.autograd.grad(loss, [dummy_w], create_graph=True)[0]
    return torch.sum(g**2)

def train_irm(
    sources, target, per_domain_train_loaders, eval_loaders, num_classes,
    epochs=25, lr=3e-4, wd=0.0, out_dir="outputs_irm",
    seed=1337, use_amp=True,
    lambda_warmup_epochs=8,    # long warmup
    lambda_after=2.0,          # stay small & steady
    grad_accum_steps=2,
    domains_per_step=2,        # 2 domains per step for stable penalty
    unfreeze_layer2_at=10,     # give more capacity mid-training
    use_swa=True, swa_start=20 # SWA last few epochs
):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = build_resnet50(num_classes).to(device)
    model = model.to(memory_format=torch.channels_last)
    model.apply(set_bn_eval)  # freeze BN stats

    opt   = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs, eta_min=1e-6)
    ce    = nn.CrossEntropyLoss()
    scaler = torch.amp.GradScaler('cuda', enabled=(use_amp and device.type=='cuda'))
    torch.set_float32_matmul_precision('medium')

    # SWA wrapper (activates after swa_start)
    swa_model = AveragedModel(model) if use_swa else None
    swa_sched = SWALR(opt, swa_lr=lr*0.5) if use_swa else None  # lr will be overridden after swa_start

    # Iterators
    domain_iters = {d: iter(per_domain_train_loaders[d]) for d in sources}
    domains = list(sources)

    best_target = 0.0
    logs = []
    os.makedirs(out_dir, exist_ok=True)

    for ep in range(1, epochs+1):
        # λ schedule
        lam = 0.0 if ep <= lambda_warmup_epochs else lambda_after

        # unfreeze layer2 later for extra capacity
        if ep == unfreeze_layer2_at:
            for p in model.layer2.parameters(): p.requires_grad = True

        model.train()
        steps = min(len(ld) for ld in per_domain_train_loaders.values())
        running_loss = 0.0
        running_pen  = 0.0
        seen = 0

        opt.zero_grad(set_to_none=True)
        for step in tqdm(range(steps), desc=f"[IRM+] Epoch {ep}/{epochs}", leave=False):
            # pick K distinct domains this step
            chosen = [domains[(step + k) % len(domains)] for k in range(domains_per_step)]
            losses, penalties, batch_sizes = [], [], []

            for d in chosen:
                try:
                    x, y = next(domain_iters[d])
                except StopIteration:
                    domain_iters[d] = iter(per_domain_train_loaders[d])
                    x, y = next(domain_iters[d])

                x = x.to(device, memory_format=torch.channels_last, non_blocking=True)
                y = y.to(device, non_blocking=True)
                w = torch.tensor(1.0, requires_grad=True, device=device)

                with torch.amp.autocast('cuda', enabled=(use_amp and device.type=="cuda")):
                    logits = model(x) * w
                    loss_d = ce(logits, y)
                pen_d = irm_penalty(loss_d, w)

                losses.append(loss_d)
                penalties.append(pen_d)
                batch_sizes.append(y.size(0))

            loss = torch.stack(losses).mean()
            penalty = torch.stack(penalties).mean()
            total = loss + lam * penalty

            scaler.scale(total / grad_accum_steps).backward()
            running_loss += total.item() * sum(batch_sizes)
            running_pen  += penalty.item() * sum(batch_sizes)
            seen += sum(batch_sizes)

            if (step + 1) % grad_accum_steps == 0:
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), 1.0)
                scaler.step(opt)
                scaler.update()
                opt.zero_grad(set_to_none=True)

        # LR schedule or SWA phase
        if use_swa and ep >= swa_start:
            swa_model.update_parameters(model)
            swa_sched.step()
        else:
            sched.step()

        # eval helper
        @torch.no_grad()
        def evaluate(m, loaders):
            m.eval(); out={}
            for name, ld in loaders.items():
                c=t=0
                for xb,yb in ld:
                    xb = xb.to(device, memory_format=torch.channels_last)
                    yb = yb.to(device)
                    p = m(xb).argmax(1)
                    c += (p==yb).sum().item(); t += yb.numel()
                out[name]=c/max(t,1)
            return out

        # If SWA just started or at the end, update BN for SWA model (needs BN stats)
        if use_swa and ep == epochs:
            # temporarily un-freeze BN for update_bn
            def set_bn_train(m):
                if isinstance(m, torch.nn.modules.batchnorm._BatchNorm): m.train()
            swa_model.apply(set_bn_train)
            torch.optim.swa_utils.update_bn(next(iter(eval_loaders.values())), swa_model, device=device)
            acc = evaluate(swa_model, eval_loaders)
        else:
            acc = evaluate(model, eval_loaders)

        src_accs = [acc[d] for d in sources]
        avg_src, worst_src = float(np.mean(src_accs)), float(min(src_accs))
        row = {
            "epoch": ep,
            "lambda": lam,
            "train_loss": running_loss / max(seen,1),
            "irm_penalty": running_pen / max(seen,1),   # <-- report this
            "target_acc": acc[target],
            "avg_source_acc": avg_src,
            "worst_source_acc": worst_src,
        }
        for d in sources+[target]:
            row[f"acc_{d}"] = acc[d]
        logs.append(row)

        print(f"[IRM+ Ep {ep:02d}] tgt={acc[target]:.3f} | src_avg={avg_src:.3f} | worst_src={worst_src:.3f} | "
              f"λ={lam:.2f} | pen={row['irm_penalty']:.4f}")

        if acc[target] > best_target:
            best_target = acc[target]
            torch.save((swa_model if (use_swa and ep>=swa_start) else model).state_dict(),
                       os.path.join(out_dir, "best_model.pt"))

    df = pd.DataFrame(logs)
    os.makedirs(out_dir, exist_ok=True)
    df.to_csv(os.path.join(out_dir, "training_log.csv"), index=False)
    print(f"\n✅ IRM+ best target ({target}) acc: {best_target:.3f}")
    return df


In [22]:
df_irm = train_irm(
    sources=SOURCES,
    target=TARGET,
    per_domain_train_loaders=per_domain_train,
    eval_loaders=eval_loaders,
    num_classes=num_classes,
    epochs=20,
    lr=3e-4,
    wd=0.0,
    out_dir="outputs_irm",
    lambda_warmup_epochs=5,
    lambda_after=2.0,
    grad_accum_steps=2,
    domains_per_step=2,
    unfreeze_layer2_at=10,
    use_swa=True, swa_start=20,
)
best_idx = df_irm["target_acc"].idxmax()
best = df_irm.iloc[best_idx]
{
    "best_epoch": int(best["epoch"]),
    "target_sketch_acc": round(float(best["target_acc"]), 4),
    "avg_source_acc": round(float(best["avg_source_acc"]), 4),
    "worst_source_acc": round(float(best["worst_source_acc"]), 4),
    "lambda_used": float(best["lambda"]),
    "irm_penalty_at_best": round(float(best["irm_penalty"]), 6),
    **{f"{d}_acc": round(float(best[f"acc_{d}"]), 4) for d in SOURCES+[TARGET]}
}


[IRM+ Ep 01] tgt=0.596 | src_avg=0.892 | worst_src=0.799 | λ=0.00 | pen=0.1277


[IRM+ Ep 02] tgt=0.724 | src_avg=0.942 | worst_src=0.897 | λ=0.00 | pen=0.0370


[IRM+ Ep 03] tgt=0.715 | src_avg=0.946 | worst_src=0.896 | λ=0.00 | pen=0.0266


[IRM+ Ep 04] tgt=0.729 | src_avg=0.969 | worst_src=0.942 | λ=0.00 | pen=0.0222


[IRM+ Ep 05] tgt=0.745 | src_avg=0.984 | worst_src=0.971 | λ=0.00 | pen=0.0144


...

[IRM+ Ep 17] tgt=0.752 | src_avg=0.979 | worst_src=0.969 | λ=2.00 | pen=0.0109


[IRM+ Ep 18] tgt=0.754 | src_avg=0.976 | worst_src=0.965 | λ=2.00 | pen=0.0127


[IRM+ Ep 19] tgt=0.734 | src_avg=0.980 | worst_src=0.971 | λ=2.00 | pen=0.0119


[IRM+ Ep 20] tgt=0.624 | src_avg=0.973 | worst_src=0.939 | λ=2.00 | pen=0.0113

✅ IRM+ best target (sketch) acc: 0.788


{'best_epoch': 7,
 'target_sketch_acc': 0.7877,
 'avg_source_acc': 0.9859,
 'worst_source_acc': 0.9731,
 'lambda_used': 0.0,
 'irm_penalty_at_best': 0.008235,
 'art_painting_acc': 0.9883,
 'cartoon_acc': 0.9731,
 'photo_acc': 0.9964,
 'sketch_acc': 0.7877}